In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('Results/RequiredDataSet.csv',header=None, names=['Booking_date','channel','property_id','room_name','room_id','check_in','check_out','room_rate','booking_status'])
#data.head()


In [2]:
data = data.filter(['Booking_date','property_id','room_rate','booking_status'], axis=1)

In [3]:
scrapyData = pd.read_csv('Results/eventData2015.csv',header=None, names=['Day','Booking_date','Event','HolidayIn'])
scrapeResult = scrapyData.dropna()
#scrapeResult.head()

In [4]:
pd.options.mode.chained_assignment = None 

In [5]:
scrapeResult['Booking_date'] =   '2015-' + scrapeResult['Booking_date'] 
#scrapeResult.head()

In [6]:
scrapeResult['Booking_date'] = pd.to_datetime(scrapeResult.Booking_date)
#scrapeResult.head()

In [7]:
del scrapeResult['HolidayIn']
del scrapeResult['Day']
# scrapeResult.head()

In [8]:
newResults = data[(data.booking_status == 'COMPLETED') | (data.booking_status == 'CONFIRMED')]
# newResults.head()

In [9]:
newResults = newResults[(newResults.Booking_date >= '2015-01-01') & (newResults.Booking_date <= '2015-12-31')]
# newResults.head()

In [10]:
newResults['Booking_date'] = newResults['Booking_date'].str.extract('(....-..-..)', expand=True)
# newResults['Booking_date'].head()

In [11]:
newResults['Booking_date'] = pd.to_datetime(newResults.Booking_date)

In [12]:
res = pd.merge(left=newResults, left_on='Booking_date',right=scrapeResult, right_on='Booking_date')

In [13]:
res = res.sort_values(by = 'Booking_date', ascending=True)
# res1 = res.drop_duplicates()
res1 = res.reset_index()
del res1['index']
res1.head()

,Booking_date,property_id,room_rate,booking_status,Event
0,2015-01-01,CH,4000.0,COMPLETED,New Years Day
1,2015-01-01,SV,16500.0,COMPLETED,New Years Day
2,2015-01-01,CL,16349.0,COMPLETED,New Years Day
3,2015-01-01,MB,48270.0,COMPLETED,New Years Day
4,2015-01-01,BB,39000.0,COMPLETED,New Years Day


In [14]:
res1['totalRevenue'] = res1.groupby(['property_id','Booking_date'])['room_rate'].transform('sum')
res2 = res1.drop_duplicates()
res2.head()

,Booking_date,property_id,room_rate,booking_status,Event,totalRevenue
0,2015-01-01,CH,4000.0,COMPLETED,New Years Day,61900.0
1,2015-01-01,SV,16500.0,COMPLETED,New Years Day,189266.0
2,2015-01-01,CL,16349.0,COMPLETED,New Years Day,182999.0
3,2015-01-01,MB,48270.0,COMPLETED,New Years Day,287280.0
4,2015-01-01,BB,39000.0,COMPLETED,New Years Day,76050.0


In [15]:
res1['room_count'] = res1.groupby(['property_id','Booking_date'])['room_rate'].transform('count')
res3 = res1
res3.head()

,Booking_date,property_id,room_rate,booking_status,Event,totalRevenue,room_count
0,2015-01-01,CH,4000.0,COMPLETED,New Years Day,61900.0,10
1,2015-01-01,SV,16500.0,COMPLETED,New Years Day,189266.0,14
2,2015-01-01,CL,16349.0,COMPLETED,New Years Day,182999.0,5
3,2015-01-01,MB,48270.0,COMPLETED,New Years Day,287280.0,4
4,2015-01-01,BB,39000.0,COMPLETED,New Years Day,76050.0,6


In [16]:
res3['ADR'] = res3.apply(lambda res3: (res3['totalRevenue'] / res3['room_count'] ) , axis=1)
res3.head()

,Booking_date,property_id,room_rate,booking_status,Event,totalRevenue,room_count,ADR
0,2015-01-01,CH,4000.0,COMPLETED,New Years Day,61900.0,10,6190.0
1,2015-01-01,SV,16500.0,COMPLETED,New Years Day,189266.0,14,13519.0
2,2015-01-01,CL,16349.0,COMPLETED,New Years Day,182999.0,5,36599.8
3,2015-01-01,MB,48270.0,COMPLETED,New Years Day,287280.0,4,71820.0
4,2015-01-01,BB,39000.0,COMPLETED,New Years Day,76050.0,6,12675.0


In [17]:
del res3['room_rate']
del res3['booking_status']
res3 = res3.drop_duplicates()
# res3

In [18]:
temp = res3['property_id'].unique()
type(temp)

numpy.ndarray

In [ ]:
for i in range(len(temp)):
    property_id = res3[(res3.property_id == temp[i])]
    property_id.to_excel('Results/GroupByPropertyID/2015/'+ temp[i] +'.xlsx')

In [19]:
# Our numerical workhorses
import numpy as np
import pandas as pd

# Import Bokeh modules for interactive plotting
import bokeh.charts
import bokeh.charts.utils
import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting

# Display graphics in this notebook
bokeh.io.output_notebook()

Loading BokehJS ...

In [20]:
import bokeh.plotting as plt
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models.formatters import DatetimeTickFormatter


In [21]:
property_id_BB = pd.read_excel('Results/GroupByPropertyID/2015/BB.xlsx')
property_id_BB = property_id_BB.reset_index()
property_id_BB = property_id_BB.sort_values(by = 'Booking_date', ascending=True)
property_id_BB1 = property_id_BB.values.T.tolist()

In [22]:
property_id_BB[['ADR']].mean()

ADR    28208.167896
dtype: float64

In [23]:
property_id_BB[['totalRevenue']].mean()

totalRevenue    240068.02988
dtype: float64

In [ ]:
property_id_BB[['ADR']].sum()

In [ ]:
property_id_BB[['totalRevenue']].sum()

In [ ]:
# property_id_BB16 = pd.read_excel('Results/GroupByPropertyID/2016/BB.xlsx')
# property_id_BB16 = property_id_BB16.reset_index()
# property_id_BB16 = property_id_BB16.sort_values(by = 'Booking_date', ascending=True)
# property_id_BB116 = property_id_BB16.values.T.tolist()

In [ ]:
# source = ColumnDataSource(
#         data=dict(
#             x=property_id_BB1[1],
#             y=property_id_BB1[4],
#             desc=property_id_BB1[3],
#             date=[x.strftime("%Y-%m-%d") for x in property_id_BB['Booking_date']]
#         )
#     )

# sourceA = ColumnDataSource(
#         data=dict(
#             x=property_id_BB116[1],
#             y=property_id_BB116[4],
#             desc=property_id_BB116[3],
#             date=[x.strftime("%Y-%m-%d") for x in property_id_BB16['Booking_date']]
#         )
#     )

# hover = HoverTool(
#         tooltips=[
#             ("Date","@date"),
#             ("Event", "@desc"),
#             ("Total Revenue","@y{1.11}")
#         ]
#     )

# p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
#            title="Graph of BookingDate vs TotalRevenue for BB (hover to view events)",x_axis_label = "Booking Date",
#                y_axis_label = "Total Revenue")
# p.left[0].formatter.use_scientific = False
# p.line('x', 'y', line_width=2, source=source, legend="2015", color="red")   
# p.line('x', 'y', line_width=2, source=sourceA, legend="2016", color="green")   
# show(p)

In [ ]:
# source = ColumnDataSource(
#         data=dict(
#             x=property_id_BB1[1],
#             y=property_id_BB1[6],
#             desc=property_id_BB1[3],
#             date=[x.strftime("%Y-%m-%d") for x in property_id_BB['Booking_date']]
#         )
#     )

# sourceA = ColumnDataSource(
#         data=dict(
#             x=property_id_BB116[1],
#             y=property_id_BB116[6],
#             desc=property_id_BB116[3],
#             date=[x.strftime("%Y-%m-%d") for x in property_id_BB16['Booking_date']]
#         )
#     )

# hover = HoverTool(
#         tooltips=[
#             ("Date","@date"),
#             ("Event", "@desc"),
#             ("Total Revenue","@y{1.11}")
#         ]
#     )

# p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
#            title="Graph of BookingDate vs ADR for BB (hover to view events)",x_axis_label = "Booking Date",
#                y_axis_label = "ADR")
# p.left[0].formatter.use_scientific = False
# p.line('x', 'y', line_width=2, source=source, legend="2015", color="red")   
# p.line('x', 'y', line_width=2, source=sourceA, legend="2016", color="green")   
# show(p)

In [24]:
source = ColumnDataSource(
        data=dict(
            x=property_id_BB1[1],
            y=property_id_BB1[4],
            desc=property_id_BB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BB['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for BB (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [25]:
source = ColumnDataSource(
        data=dict(
            x=property_id_BB1[1],
            y=property_id_BB1[6],
            desc=property_id_BB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BB['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for BB (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_BG = pd.read_excel('Results/GroupByPropertyID/2015/BG.xlsx')
property_id_BG = property_id_BG.reset_index()
property_id_BG = property_id_BG.sort_values(by = 'Booking_date', ascending=True)
property_id_BG1 = property_id_BG.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_BG1[1],
            y=property_id_BG1[4],
            desc=property_id_BG1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BG['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for BG (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_BG1[1],
            y=property_id_BG1[6],
            desc=property_id_BG1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BG['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for BG (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_CH = pd.read_excel('Results/GroupByPropertyID/2015/CH.xlsx')
property_id_CH = property_id_CH.reset_index()
property_id_CH = property_id_CH.sort_values(by = 'Booking_date', ascending=True)
property_id_CH1 = property_id_CH.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_CH1[1],
            y=property_id_CH1[4],
            desc=property_id_CH1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_CH['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for CH (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_CH1[1],
            y=property_id_CH1[6],
            desc=property_id_CH1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_CH['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for CH (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_CL = pd.read_excel('Results/GroupByPropertyID/2015/CL.xlsx')
property_id_CL = property_id_CL.reset_index()
property_id_CL = property_id_CL.sort_values(by = 'Booking_date', ascending=True)
property_id_CL1 = property_id_CL.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_CL1[1],
            y=property_id_CL1[4],
            desc=property_id_CL1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_CL['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for CL (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_CL1[1],
            y=property_id_CL1[6],
            desc=property_id_CL1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_CL['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for CL (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_CP = pd.read_excel('Results/GroupByPropertyID/2015/CP.xlsx')
property_id_CP = property_id_CP.reset_index()
property_id_CP = property_id_CP.sort_values(by = 'Booking_date', ascending=True)
property_id_CP1 = property_id_CP.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_CP1[1],
            y=property_id_CP1[4],
            desc=property_id_CP1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_CP['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for CP (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_EB = pd.read_excel('Results/GroupByPropertyID/2015/EB.xlsx')
property_id_EB = property_id_EB.reset_index()
property_id_EB = property_id_EB.sort_values(by = 'Booking_date', ascending=True)
property_id_EB1 = property_id_EB.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_EB1[1],
            y=property_id_EB1[4],
            desc=property_id_EB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_EB['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for EB (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_EB1[1],
            y=property_id_EB1[6],
            desc=property_id_EB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_EB['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for EB (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_KK = pd.read_excel('Results/GroupByPropertyID/2015/KK.xlsx')
property_id_KK = property_id_KK.reset_index()
property_id_KK = property_id_KK.sort_values(by = 'Booking_date', ascending=True)
property_id_KK1 = property_id_KK.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_KK1[1],
            y=property_id_KK1[4],
            desc=property_id_KK1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_KK['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for KK (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_KK1[1],
            y=property_id_KK1[6],
            desc=property_id_KK1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_KK['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for KK (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_KR = pd.read_excel('Results/GroupByPropertyID/2015/KR.xlsx')
property_id_KR = property_id_KR.reset_index()
property_id_KR = property_id_KR.sort_values(by = 'Booking_date', ascending=True)
property_id_KR1 = property_id_KR.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_KR1[1],
            y=property_id_KR1[4],
            desc=property_id_KR1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_KR['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for KR (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_KR1[1],
            y=property_id_KR1[6],
            desc=property_id_KR1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_KR['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for KR (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_MB = pd.read_excel('Results/GroupByPropertyID/2015/MB.xlsx')
property_id_MB = property_id_MB.reset_index()
property_id_MB = property_id_MB.sort_values(by = 'Booking_date', ascending=True)
property_id_MB1 = property_id_MB.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_MB1[1],
            y=property_id_MB1[4],
            desc=property_id_MB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_MB['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for MB (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_MB1[1],
            y=property_id_MB1[6],
            desc=property_id_MB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_MB['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for MB (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_MM = pd.read_excel('Results/GroupByPropertyID/2015/MM.xlsx')
property_id_MM = property_id_MM.reset_index()
property_id_MM = property_id_MM.sort_values(by = 'Booking_date', ascending=True)
property_id_MM1 = property_id_MM.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_MM1[1],
            y=property_id_MM1[4],
            desc=property_id_MM1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_MM['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for MM (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_MM1[1],
            y=property_id_MM1[6],
            desc=property_id_MM1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_MM['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for MM (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_MP = pd.read_excel('Results/GroupByPropertyID/2015/MP.xlsx')
property_id_MP = property_id_MP.reset_index()
property_id_MP = property_id_MP.sort_values(by = 'Booking_date', ascending=True)
property_id_MP1 = property_id_MP.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_MP1[1],
            y=property_id_MP1[4],
            desc=property_id_MP1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_MP['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for MP (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_MP1[1],
            y=property_id_MP1[6],
            desc=property_id_MP1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_MP['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for MP (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_SC = pd.read_excel('Results/GroupByPropertyID/2015/SC.xlsx')
property_id_SC = property_id_SC.reset_index()
property_id_SC = property_id_SC.sort_values(by = 'Booking_date', ascending=True)
property_id_SC1 = property_id_SC.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_SC1[1],
            y=property_id_SC1[4],
            desc=property_id_SC1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_SC['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for SC (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_SC1[1],
            y=property_id_SC1[6],
            desc=property_id_SC1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_SC['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for SC (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_SS = pd.read_excel('Results/GroupByPropertyID/2015/SS.xlsx')
property_id_SS = property_id_SS.reset_index()
property_id_SS = property_id_SS.sort_values(by = 'Booking_date', ascending=True)
property_id_SS1 = property_id_SS.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_SS1[1],
            y=property_id_SS1[4],
            desc=property_id_SS1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_SS['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for SS (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_SS1[1],
            y=property_id_SS1[6],
            desc=property_id_SS1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_SS['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for SS (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_SV = pd.read_excel('Results/GroupByPropertyID/2015/SV.xlsx')
property_id_SV = property_id_SV.reset_index()
property_id_SV = property_id_SV.sort_values(by = 'Booking_date', ascending=True)
property_id_SV1 = property_id_SV.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_SV1[1],
            y=property_id_SV1[4],
            desc=property_id_SV1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_SV['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for SV (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_SV1[1],
            y=property_id_SV1[6],
            desc=property_id_SV1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_SV['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for SV (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_VM = pd.read_excel('Results/GroupByPropertyID/2015/VM.xlsx')
property_id_VM = property_id_VM.reset_index()
property_id_VM = property_id_VM.sort_values(by = 'Booking_date', ascending=True)
property_id_VM1 = property_id_VM.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_VM1[1],
            y=property_id_VM1[4],
            desc=property_id_VM1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_VM['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for VM (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_VM1[1],
            y=property_id_VM1[6],
            desc=property_id_VM1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_VM['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for VM (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
property_id_VV = pd.read_excel('Results/GroupByPropertyID/2015/VV.xlsx')
property_id_VV = property_id_VV.reset_index()
property_id_VV = property_id_VV.sort_values(by = 'Booking_date', ascending=True)
property_id_VV1 = property_id_VV.values.T.tolist()


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_VV1[1],
            y=property_id_VV1[4],
            desc=property_id_VV1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_VV['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of BookingDate vs TotalRevenue for VV (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)


In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_VV1[1],
            y=property_id_VV1[6],
            desc=property_id_VV1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_VV['Booking_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of Booking Date vs ADR for VV (hover to view events)",x_axis_label = "Booking Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)
